## How to apply JEC with coffea?
The boxes below take the JME-provided files to create an object that calibrates AK8 PUPPI jets

In [1]:
import importlib.resources
import contextlib
from coffea.lookup_tools import extractor
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory, CorrectedMETFactory


In [2]:
#Adapted from https://github.com/nsmith-/boostedhiggs/blob/cc/boostedhiggs/build_jec.py
jec_name_map = {
    'JetPt': 'pt',
    'JetMass': 'mass',
    'JetEta': 'eta',
    'JetA': 'area',
    'ptGenJet': 'pt_gen',
    'ptRaw': 'pt_raw',
    'massRaw': 'mass_raw',
    'Rho': 'event_rho',
    'METpt': 'pt',
    'METphi': 'phi',
    'JetPhi': 'phi',
    'UnClusteredEnergyDeltaX': 'MetUnclustEnUpDeltaX',
    'UnClusteredEnergyDeltaY': 'MetUnclustEnUpDeltaY',
}


def jet_factory_factory(files,path):
    ext = extractor()
    with contextlib.ExitStack() as stack:
        # this would work even in zipballs but since extractor keys on file extension and
        # importlib make a random tempfile, it won't work. coffea needs to enable specifying the type manually
        # for now we run this whole module as $ python -m boostedhiggs.build_jec boostedhiggs/data/jec_compiled.pkl.gz
        # so the compiled value can be loaded using the importlib tool in corrections.py
        real_files = [path+f for f in files]
        ext.add_weight_sets([f"* * {file}" for file in real_files])
        ext.finalize()

    jec_stack = JECStack(ext.make_evaluator())
    return CorrectedJetsFactory(jec_name_map, jec_stack)


The .jec, .junc, .jr and .jersf tell the `JECStack` what is the nature of the payload. From the name of the file, it figures out which correction it is and applies them in proper order (L1, L2, L3, L2L3) 

In [3]:
#UL 
#JEC from https://twiki.cern.ch/twiki/bin/view/CMS/JECDataMC#Recommended_for_MC
#JER from https://github.com/cms-jet/JRDatabase/tree/master/tarballs
fatjet_factory = {    
    "2016APVmc": jet_factory_factory(
        files=[
            "Summer19UL16APV_V7_MC_L1FastJet_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16APV_V7_MC_L2Relative_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16APV_V7_MC_L2L3Residual_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16APV_V7_MC_L2Residual_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16APV_V7_MC_L3Absolute_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16APV_V7_MC_Uncertainty_AK8PFPuppi.junc.txt.gz",
            "Summer19UL16APV_V7_MC_UncertaintySources_AK8PFPuppi.junc.txt.gz",
            "Summer20UL16APV_JRV3_MC_PtResolution_AK8PFPuppi.jr.txt.gz",
            "Summer20UL16APV_JRV3_MC_SF_AK8PFPuppi.jersf.txt.gz"
        ],path="../data/jec/2016APV/"
    ),
        "2016mc": jet_factory_factory(
        files=[
            "Summer19UL16_V7_MC_L1FastJet_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16_V7_MC_L2Relative_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16_V7_MC_L2L3Residual_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16_V7_MC_L2Residual_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16_V7_MC_L3Absolute_AK8PFPuppi.jec.txt.gz",
            "Summer19UL16_V7_MC_Uncertainty_AK8PFPuppi.junc.txt.gz",
            "Summer19UL16_V7_MC_UncertaintySources_AK8PFPuppi.junc.txt.gz",
            "Summer20UL16_JRV3_MC_PtResolution_AK8PFPuppi.jr.txt.gz",
            "Summer20UL16_JRV3_MC_SF_AK8PFPuppi.jersf.txt.gz"
        ],path="../data/jec/2016/"
    ),
        "2017mc": jet_factory_factory(
        files=[
            "Summer19UL17_V5_MC_L1FastJet_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_L2Relative_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_L2L3Residual_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_L2Residual_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_L3Absolute_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_Uncertainty_AK8PFPuppi.junc.txt.gz",
            "Summer19UL17_V5_MC_UncertaintySources_AK8PFPuppi.junc.txt.gz",
            "Summer19UL17_JRV3_MC_PtResolution_AK8PFPuppi.jr.txt.gz",
            "Summer19UL17_JRV3_MC_SF_AK8PFPuppi.jersf.txt.gz"
        ],path="../data/jec/2017/"
    ),
        "2017mcNoL2Res": jet_factory_factory(
        files=[
            "Summer19UL17_V5_MC_L1FastJet_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_L2Relative_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_L2L3Residual_AK8PFPuppi.jec.txt.gz",
            #"Summer19UL17_V5_MC_L2Residual_AK8PFPuppi.jec.txt.gz", #Looks like L2Res does not change the result, included in L2L23 Residual?
            "Summer19UL17_V5_MC_L3Absolute_AK8PFPuppi.jec.txt.gz",
            "Summer19UL17_V5_MC_Uncertainty_AK8PFPuppi.junc.txt.gz",
            "Summer19UL17_V5_MC_UncertaintySources_AK8PFPuppi.junc.txt.gz",
            "Summer19UL17_JRV3_MC_PtResolution_AK8PFPuppi.jr.txt.gz",
            "Summer19UL17_JRV3_MC_SF_AK8PFPuppi.jersf.txt.gz"
        ],path="../data/jec/2017/"
    ),
        "2018mc": jet_factory_factory(
        files=[
        "Summer19UL18_V5_MC_L1FastJet_AK8PFPuppi.jec.txt.gz",
        "Summer19UL18_V5_MC_L2Relative_AK8PFPuppi.jec.txt.gz",
        "Summer19UL18_V5_MC_L2L3Residual_AK8PFPuppi.jec.txt.gz",
        "Summer19UL18_V5_MC_L2Residual_AK8PFPuppi.jec.txt.gz",
        "Summer19UL18_V5_MC_L3Absolute_AK8PFPuppi.jec.txt.gz",
        "Summer19UL18_V5_MC_Uncertainty_AK8PFPuppi.junc.txt.gz",
        "Summer19UL18_V5_MC_UncertaintySources_AK8PFPuppi.junc.txt.gz",
        "Summer19UL18_JRV2_MC_PtResolution_AK8PFPuppi.jr.txt.gz",
        "Summer19UL18_JRV2_MC_SF_AK8PFPuppi.jersf.txt.gz"

        ],path="../data/jec/2018/"
    )
}


if __name__ == "__main__":
    import gzip
    import cloudpickle
    
    with gzip.open("jme_UL_pickled.pkl", "wb") as fout:
        cloudpickle.dump(
            {
                "fatjet_factory": fatjet_factory,
            },
            fout
        )
    print("Done")

Done


## Using the calibrator object
The above box stored a "fatjet_factory" dictionary that contains MC calibrations separated per year.<br>
We will now load it and apply it to the FaJets collection

In [4]:
import gzip
import cloudpickle
with gzip.open("jme_UL_pickled.pkl") as fin:
    jmestuff = cloudpickle.load(fin)
    fatjet_factory = jmestuff["fatjet_factory"]

In [5]:
def add_jec_variables(jets, event_rho):
    jets["pt_raw"] = (1 - jets.rawFactor)*jets.pt
    jets["mass_raw"] = (1 - jets.rawFactor)*jets.mass
    jets["pt_gen"] = ak.values_astype(ak.fill_none(jets.matched_gen.pt, 0), np.float32)
    jets["event_rho"] = ak.broadcast_arrays(event_rho, jets.pt)[0]
    return jets

In [6]:
import numpy as np
import awkward as ak
import ROOT
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
from hist import Hist

Welcome to JupyROOT 6.24/07


In [7]:
events = NanoEventsFactory.from_root("/STORE/matej/H3_skims/2017/TTbarHadronic/DBADF3D8-7C75-F74B-99AF-7AF0D41083BE.root",schemaclass=NanoAODSchema,metadata={"dataset":""},entry_stop=None).events()

In [8]:
jec_cache = {}
fatjets    = events.FatJet
fatjets_calib = fatjet_factory["2017mc"].build(add_jec_variables(events.FatJet, events.fixedGridRhoFastjetAll), jec_cache)
fatjets_calib_no_L2Res = fatjet_factory["2017mcNoL2Res"].build(add_jec_variables(events.FatJet, events.fixedGridRhoFastjetAll), jec_cache)

/tmp/ipykernel_3365042/3890649366.py:4: DeprecationWarning: In version 1.7.0 (target date: 2021-10-01), this will be changed.

To raise these warnings as errors (and get stack traces to find out where they're called), run

    import warnings
    warnings.filterwarnings("error", module="awkward.*")

after the first `import awkward` or use `@pytest.mark.filterwarnings("error:::awkward.*")` in pytest.

Issue: ak.fill_none needs an explicit `axis` because the default will change to `axis=-1`.
  jets["pt_gen"] = ak.values_astype(ak.fill_none(jets.matched_gen.pt, 0), np.float32)


In [9]:
print(fatjets.pt[0][0])
print(fatjets_calib.pt[0][0])
print(fatjets_calib.JES_jes.up.pt[0][0])
print(fatjets_calib.JES_jes.down.pt[0][0])
print(fatjets_calib.JER.down.pt[0][0])
print(fatjets_calib.JER.up.pt[0][0])

332.25
330.85260009765625
332.6395263671875
329.065673828125
331.6187744140625
330.08782958984375


Looks like there is some difference between pt in the NanoAOD and the calibrated pt. Either the MC dataset does not contain the latest and greatest JEC or I messed up the implementation.<br>
Also, by comparing the output of boxes above and below, I conclude that L2Res payload does not contribute to JEC. This makes sense because L2Res is part of L2L3Res that we already apply.

In [10]:
print(fatjets_calib_no_L2Res.pt[0][0])
print(fatjets_calib_no_L2Res.JES_jes.up.pt[0][0])
print(fatjets_calib_no_L2Res.JES_jes.down.pt[0][0])
print(fatjets_calib_no_L2Res.JER.down.pt[0][0])
print(fatjets_calib_no_L2Res.JER.up.pt[0][0])

330.85260009765625
332.6395263671875
329.065673828125
331.6187744140625
330.08782958984375
